Используя RandomForestClassifier и еще 2 метода МО предсказать будущий ABC класс на основе любых фичей

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# почему-то так не работает FILEPATH = 'DataSet_w_NA.xlsx'
path = (r'C:\Users\Юлия\pythonjul\MISIS-master\MISIS-master\notebooks\season_2\DataSet_w_NA.xlsx')

df = pd.read_excel(path, sheet_name="Испорченные факты").dropna()
df.head()

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.20,1,7,3
1,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.80,1,4,8
2,2014-100328,Нет,2015-07-28 00:00:00,2015-08-03 00:00:00,Стандарт,CLNT0000342,PROD0000187,1964.0,1.0,0.2,662.85,1,4,12
3,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001095,1184.0,2.0,0.2,414.40,2,12,7
4,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001616,9504.0,3.0,0.2,3445.20,2,12,3


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9993 entries, 0 to 10292
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Номер заказа            9993 non-null   object 
 1   Возврат                 9993 non-null   object 
 2   Дата заказа             9993 non-null   object 
 3   Дата отправки           9993 non-null   object 
 4   Режим отправки          9993 non-null   object 
 5   Факты.Контрагент ID     9993 non-null   object 
 6   Факты.Товар ID          9993 non-null   object 
 7   Продажи, руб            9993 non-null   float64
 8   Продажи, шт             9993 non-null   float64
 9   Скидка                  9993 non-null   float64
 10  Маржинальная прибыль    9993 non-null   float64
 11  Повторение заказа       9993 non-null   int64  
 12  Повторение контрагента  9993 non-null   int64  
 13  Повторение товара       9993 non-null   int64  
dtypes: float64(4), int64(3), object(7)
memo

In [13]:
df.isnull().values.any() #проверим отсутсвие неопределенностей в датасете

False

In [14]:
df.isnull().sum().sum() #проверим отсутсвие нулей в датасете

0

In [15]:
df.isna().sum().sum() #проверим отсутсвие неопределенностей в датасете

0

In [50]:
df =pd.pivot_table(df, values=['Продажи, руб', 'Продажи, шт','Повторение заказа','Маржинальная прибыль','Повторение товара'], index=["Факты.Товар ID"],
                     aggfunc={'Продажи, шт': [np.median, np.sum],
                              'Продажи, руб': np.sum,
                              'Повторение заказа': np.sum,
                              'Маржинальная прибыль': np.sum
                        
                               })

In [51]:
df.head()

Маржинальная прибыль Повторение заказа Продажи, руб  \
                                sum               sum          sum   
Факты.Товар ID                                                       
PROD0000001               -28160.70                 6     631978.5   
PROD0000002               963721.00                32    5318764.0   
PROD0000003                32200.55                18      76755.0   
PROD0000004              -266380.65                17    3621384.0   
PROD0000005                31874.40                18     150984.0   

               Продажи, шт        
                    median   sum  
Факты.Товар ID                    
PROD0000001            2.5  12.0  
PROD0000002            3.5  51.0  
PROD0000003            2.0  23.0  
PROD0000004            4.0  29.0  
PROD0000005            5.0  24.0

In [52]:
newname=df.columns.map('_'.join)
newname

Index(['Маржинальная прибыль_sum', 'Повторение заказа_sum', 'Продажи, руб_sum',
       'Продажи, шт_median', 'Продажи, шт_sum'],
      dtype='object')

In [53]:
df.columns=newname

In [30]:
df.head()

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum"
Факты.Товар ID,,,,,
PROD0000001,-28160.70,6,631978.5,2.5,12.0
PROD0000002,963721.00,32,5318764.0,3.5,51.0
PROD0000003,32200.55,18,76755.0,2.0,23.0
PROD0000004,-266380.65,17,3621384.0,4.0,29.0
PROD0000005,31874.40,18,150984.0,5.0,24.0


In [31]:
df=df.reset_index()
df.head()

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum"
0,PROD0000001,-28160.70,6,631978.5,2.5,12.0
1,PROD0000002,963721.00,32,5318764.0,3.5,51.0
2,PROD0000003,32200.55,18,76755.0,2.0,23.0
3,PROD0000004,-266380.65,17,3621384.0,4.0,29.0
4,PROD0000005,31874.40,18,150984.0,5.0,24.0


In [54]:
total_sale=df['Продажи, руб_sum'].sum()
df['Доля']=df['Продажи, руб_sum']/total_sale * 100
df = df.sort_values(by=('Продажи, руб_sum'), ascending=False)
df=df.assign(sum_d=df['Доля'].cumsum())
df.head()

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d
Факты.Товар ID,,,,,,,
PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845
PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071
PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673
PROD0000488,2.328306e-10,15,10935288.0,5.0,39.0,0.952170,5.814843
PROD0001187,1.116753e+06,33,9911739.5,3.0,37.0,0.863046,6.677889


In [55]:
df.loc[(df['sum_d'] <= 80), 'ABC'] = 'A'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'ABC'] = 'B'
df.loc[(df['sum_d'] > 95), 'ABC'] = 'C'
df

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC
Факты.Товар ID,,,,,,,,
PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845,A
PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071,A
PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673,A
PROD0000488,2.328306e-10,15,10935288.0,5.0,39.0,0.952170,5.814843,A
PROD0001187,1.116753e+06,33,9911739.5,3.0,37.0,0.863046,6.677889,A
...,...,...,...,...,...,...,...,...
PROD0001871,1.578500e+03,1,3850.0,2.0,2.0,0.000335,99.999089,C
PROD0001887,1.193400e+03,1,3536.0,2.0,2.0,0.000308,99.999396,C
PROD0001779,1.555200e+03,3,3240.0,1.0,1.0,0.000282,99.999679,C


In [117]:
df['Стоимость, руб'] = df['Продажи, руб_sum']/df['Продажи, шт_sum']
df['Продажи в следующем периоде']= (df['Продажи, шт_sum'] + df['Продажи, шт_median']) *df['Стоимость, руб']
total_sale_next=df['Продажи в следующем периоде'].sum()
df['Доля_будущая']=df['Продажи в следующем периоде']/total_sale_next * 100
df = df.sort_values(by=('Продажи в следующем периоде'), ascending=False)
df=df.assign(sum_d_next=df['Доля_будущая'].cumsum())
df['Доля_будущая'].sum()

df.loc[(df['sum_d_next'] <= 80), 'ABC_next'] = 'A'
df.loc[(df['sum_d_next'] > 80) & (df['sum_d_next'] <= 95), 'ABC_next'] = 'B'
df.loc[(df['sum_d_next'] > 95), 'ABC_next'] = 'C'

df.loc[(df['ABC'] != df['ABC_next']), 'Изменение класса'] = 1
df.loc[(df['ABC'] == df['ABC_next']), 'Изменение класса'] = 0


In [57]:
df.loc[(df['sum_d'] <= 80), 'class'] = '0'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'class'] = '1'
df.loc[(df['sum_d'] > 95), 'class'] = '2'
df

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC,"Стоимость, руб",Продажи в следующем периоде,Доля_будущая,sum_d_next,ABC_next,Изменение класса,class
Факты.Товар ID,,,,,,,,,,,,,,,
PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845,A,1.539996e+06,3.695989e+07,2.691941,2.691941,A,0.0,0
PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673,A,1.886540e+06,2.263848e+07,1.648854,4.340795,A,0.0,0
PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071,A,4.427965e+05,1.505508e+07,1.096523,5.437318,A,0.0,0
PROD0000488,2.328306e-10,15,10935288.0,5.0,39.0,0.952170,5.814843,A,2.803920e+05,1.233725e+07,0.898572,6.335891,A,0.0,0
PROD0001657,2.047488e+06,6,9187447.5,4.0,12.0,0.799980,9.126346,A,7.656206e+05,1.224993e+07,0.892213,7.228103,A,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PROD0001257,1.050000e+03,12,4200.0,2.5,5.0,0.000366,99.997704,C,8.400000e+02,6.300000e+03,0.000459,99.998575,C,0.0,2
PROD0001119,2.220000e+02,13,4884.0,1.5,6.0,0.000425,99.996103,C,8.140000e+02,6.105000e+03,0.000445,99.999020,C,0.0,2
PROD0000808,1.702000e+03,2,4048.0,2.5,5.0,0.000352,99.998056,C,8.096000e+02,6.072000e+03,0.000442,99.999462,C,0.0,2


In [59]:
df.loc[(df['sum_d_next'] <= 80), 'class_next'] = '0'
df.loc[(df['sum_d_next'] > 80) & (df['sum_d_next'] <= 95), 'class_next'] = '1'
df.loc[(df['sum_d_next'] > 95), 'class_next'] = '2'
df

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC,"Стоимость, руб",Продажи в следующем периоде,Доля_будущая,sum_d_next,ABC_next,Изменение класса,class,class_next
Факты.Товар ID,,,,,,,,,,,,,,,,
PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845,A,1.539996e+06,3.695989e+07,2.691941,2.691941,A,0.0,0,0
PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673,A,1.886540e+06,2.263848e+07,1.648854,4.340795,A,0.0,0,0
PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071,A,4.427965e+05,1.505508e+07,1.096523,5.437318,A,0.0,0,0
PROD0000488,2.328306e-10,15,10935288.0,5.0,39.0,0.952170,5.814843,A,2.803920e+05,1.233725e+07,0.898572,6.335891,A,0.0,0,0
PROD0001657,2.047488e+06,6,9187447.5,4.0,12.0,0.799980,9.126346,A,7.656206e+05,1.224993e+07,0.892213,7.228103,A,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PROD0001257,1.050000e+03,12,4200.0,2.5,5.0,0.000366,99.997704,C,8.400000e+02,6.300000e+03,0.000459,99.998575,C,0.0,2,2
PROD0001119,2.220000e+02,13,4884.0,1.5,6.0,0.000425,99.996103,C,8.140000e+02,6.105000e+03,0.000445,99.999020,C,0.0,2,2
PROD0000808,1.702000e+03,2,4048.0,2.5,5.0,0.000352,99.998056,C,8.096000e+02,6.072000e+03,0.000442,99.999462,C,0.0,2,2


In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score


In [115]:
df['ABC'] = df['ABC'].astype('category')
df['class'] = df['ABC'].cat.codes 
dict_of_classes = dict(enumerate(df["ABC"].cat.categories))

def RandomForest_model(features, predicted_values):
    X = df[features] #фичи
    Y = df['class'] #предсказываемые значения
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state = 2020, stratify=Y) #разбиваем выборку для обучения и тестирования
   
    scaler = StandardScaler() #стадартизируем выборку
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    y_train = np.array(y_train)
    
    RFC = RandomForestClassifier(min_samples_leaf=50, n_estimators=150, bootstrap=True, oob_score=True, n_jobs=-1, max_features='auto')
    RFC.fit(X_train_scaled, y_train)
    
    print('RFC оценка: ', RFC.score(X_train_scaled, y_train))
    
    y_pred = RFC.predict(X_test_scaled)
    accuracy_score(y_test, y_pred)
    print('Accuracy: ', round(accuracy_score(y_test, y_pred),3)*100, '%')
    
    predict_data = [predicted_values]
    predict_data_scaled = scaler.transform(predict_data)
    
    new_class = RFC.predict(predict_data)
    
    if new_class[0] in dict_of_classes.keys():
        class_name = dict_of_classes[new_class[0]]
    print('Предсказанный класс: ', class_name)

In [103]:
RandomForest_model(features = ['Стоимость, руб', 'Продажи в следующем периоде', 'Доля_будущая'], predicted_values = [406, 1624, 0.000118])

RFC оценка 0.9724647414372062
Accuracy 96.2 %
Предсказанный класс:  A


In [116]:
from sklearn.neighbors import KNeighborsClassifier
def KNN_model(features, predicted_values):
    X = df[features] #фичи
    Y = df['class'] #предсказываемые значения
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state = 2020, stratify=Y) #разбиваем выборку для обучения и тестирования
   
    scaler = StandardScaler() #стадартизируем выборку
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    y_train = np.array(y_train)
    
    knn_model = KNeighborsClassifier()
    knn_model.fit(X_train_scaled, y_train)
    print('SVM оценка:', round(knn_model.score(X_train_scaled, y_train)))
                              
    y_pred = knn_model.predict(X_test_scaled)
    score = accuracy_score(y_test, y_pred)
    
    print('Accuracy: ', round(score, 4)*100 , '%')
    
    predict_data = [predicted_values]
    predict_data_scaled = scaler.transform(predict_data)
    new_class = knn_model.predict(predict_data)
    
    if new_class[0] in dict_of_classes.keys():
        class_name = dict_of_classes[new_class[0]]
    print('Предсказанный класс: ', class_name)

In [114]:
KNN_model(features = ['Стоимость, руб', 'Продажи в следующем периоде', 'Доля_будущая'], predicted_values = [406, 1624, 0.000118])

SVM оценка 1
Accuracy 96.78 %
Предсказанный класс:  A
